In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import petab
import pypesto
import pypesto.petab


In [2]:
# rng seed
np.random.seed(500)

Import petab files and define the problem

In [3]:
petab_yaml = 'petab_files/age_of_infection_model.yaml'
petab.validate(petab_yaml)
petab_problem = petab.Problem.from_yaml(petab_yaml)

problem = pypesto.petab.PetabImporter(
        petab_problem,
        hierarchical=False,
        model_name=f"Age_of_Infection_Model",
    ).create_problem(force_compile=True, verbose=False)
problem.objective.amici_model.setAllStatesNonNegative()

Compiling amici model to folder /Users/yuhongliu/Documents/OV/model/age_of_infection_model/amici_models/0.27.0/Age_of_Infection_Model.


# Gradient Check

In [4]:
# obtain random startpoints
startpoints = problem.get_startpoints(n_starts=2)

In [5]:
problem.objective.check_grad(
    x = startpoints[0],
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
rho,8311.124286,8345.590983,8276.392723,8310.991853,69.198260,0.132433,1.593473e-05
kappa,5964.704150,5987.552814,5947.242734,5967.397774,40.310080,2.693624,4.513901e-04
psi,-207.244997,-219.736412,-196.191357,-207.963885,23.545055,0.718888,3.456792e-03
phi,-574.908675,-587.731662,-561.673381,-574.702522,26.058281,0.206153,3.587130e-04
beta,-0.000094,-13.393312,13.397471,0.002079,26.790783,0.002173,1.039971e+00
alpha,-2.173200,-14.198518,8.798260,-2.700129,22.996779,0.526929,1.951504e-01
delta,27.638521,15.992469,41.587793,28.790131,25.595324,1.151610,4.000014e-02
sigma_a,-6748.100790,-6761.428395,-6734.773305,-6748.100850,26.655089,0.000060,8.855918e-09
sigma_b,-3048.476873,-3061.892249,-3035.060152,-3048.476200,26.832097,0.000673,2.207325e-07


In [6]:
problem.objective.check_grad(
    x = problem.get_reduced_vector(startpoints[0]),
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
rho,8311.124286,8345.590983,8276.392723,8310.991853,69.198260,0.132433,1.593473e-05
kappa,5964.704150,5987.552814,5947.242734,5967.397774,40.310080,2.693624,4.513901e-04
psi,-207.244997,-219.736412,-196.191357,-207.963885,23.545055,0.718888,3.456792e-03
phi,-574.908675,-587.731662,-561.673381,-574.702522,26.058281,0.206153,3.587130e-04
beta,-0.000094,-13.393312,13.397471,0.002079,26.790783,0.002173,1.039971e+00
alpha,-2.173200,-14.198518,8.798260,-2.700129,22.996779,0.526929,1.951504e-01
delta,27.638521,15.992469,41.587793,28.790131,25.595324,1.151610,4.000014e-02
sigma_a,-6748.100790,-6761.428395,-6734.773305,-6748.100850,26.655089,0.000060,8.855918e-09
sigma_b,-3048.476873,-3061.892249,-3035.060152,-3048.476200,26.832097,0.000673,2.207325e-07


In [7]:
gc = problem.objective.check_grad_multi_eps(
    x=startpoints[0],
    verbosity=0,
    label='rel_err',  # default
)
gc

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
rho,8311.124286,8309.378309,8312.889634,8311.133972,3.511325,9.685193e-03,1.165327e-06,1.000000e-03
kappa,5964.704150,5967.868590,5962.088125,5964.978358,5.780466,2.742076e-01,4.596957e-05,1.000000e-03
psi,-207.244997,-207.507447,-206.980502,-207.243974,0.526945,1.022514e-03,4.933887e-06,1.000000e-03
phi,-574.908675,-576.069756,-573.766012,-574.917884,2.303744,9.209621e-03,1.601905e-05,1.000000e-03
beta,-0.000094,-0.001445,0.001256,-0.000094,0.002701,8.930820e-07,8.939259e-06,1.000000e-01
alpha,-2.173200,-2.302367,-2.021702,-2.162034,0.280665,1.116574e-02,5.166852e-03,1.000000e-03
delta,27.638521,27.553397,27.750685,27.652041,0.197288,1.351967e-02,4.889036e-04,1.000000e-03
sigma_a,-6748.100790,-8087.393644,-5408.808033,-6748.100839,2678.585611,4.846016e-05,7.181303e-09,1.000000e-07
sigma_b,-3048.476873,-136977.822876,130880.869619,-3048.476628,267858.692496,2.447758e-04,8.029448e-08,1.000000e-09


In [8]:
def highlight_value_above_threshold(x, threshold=10):
    return ['color: darkorange' if xi > threshold else None for xi in x]

gc.style.apply(
    highlight_value_above_threshold, subset=["fd_err"],
).background_gradient(
    cmap=sns.light_palette("purple", as_cmap=True), subset=["abs_err"],
).background_gradient(
    cmap=sns.light_palette("red", as_cmap=True), subset=["rel_err"],
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
rho,8311.124286,8309.378309,8312.889634,8311.133972,3.511325,0.009685,0.000001,0.001000
kappa,5964.704150,5967.868590,5962.088125,5964.978358,5.780466,0.274208,0.000046,0.001000
psi,-207.244997,-207.507447,-206.980502,-207.243974,0.526945,0.001023,0.000005,0.001000
phi,-574.908675,-576.069756,-573.766012,-574.917884,2.303744,0.009210,0.000016,0.001000
beta,-0.000094,-0.001445,0.001256,-0.000094,0.002701,0.000001,0.000009,0.100000
alpha,-2.173200,-2.302367,-2.021702,-2.162034,0.280665,0.011166,0.005167,0.001000
delta,27.638521,27.553397,27.750685,27.652041,0.197288,0.013520,0.000489,0.001000
sigma_a,-6748.100790,-8087.393644,-5408.808033,-6748.100839,2678.585611,0.000048,0.000000,0.000000
sigma_b,-3048.476873,-136977.822876,130880.869619,-3048.476628,267858.692496,0.000245,0.000000,0.000000
